In [41]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [42]:
train = pd.read_csv(r"C:\Users\aayus\Desktop\Kaggle\Real or Fake\train.csv")
test = pd.read_csv(r"C:\Users\aayus\Desktop\Kaggle\Real or Fake\test.csv")
submission = pd.read_csv(r"C:\Users\aayus\Desktop\Kaggle\Real or Fake\sample_submission.csv")

In [43]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop

In [44]:
from nltk.tokenize import word_tokenize

In [45]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [46]:
for i in range(len(train)):
    train['text'][i] = word_tokenize(train['text'][i])
print(type(train['text'][0]))
print(train['text'][0])

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'list'>
['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']


In [47]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [48]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [49]:
for i in range(len(train)):
    train['text'][i] = [w for w in train['text'][i] if w not in stop_words]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [51]:
for i in range(len(train)):
    train['text'][i] = [ps.stem(w) for w in train['text'][i]]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [53]:
for i in range(len(train)):
    train['text'][i] = [lemmatizer.lemmatize(w) for w in train['text'][i]]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
maxlen = -1
for i in range(len(train)):
    maxlen = max(len(train['text'][i]), maxlen)
print(maxlen)

70


In [55]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [56]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train['text'])
sequences = tok.texts_to_sequences(train['text'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [57]:
train['text']

0       [our, deed, reason, #, earthquak, may, allah, ...
1         [forest, fire, near, La, rong, sask, ., canada]
2       [all, resid, ask, 'shelter, place, ', notifi, ...
3       [13,000, peopl, receiv, #, wildfir, evacu, ord...
4       [just, got, sent, photo, rubi, #, alaska, smok...
                              ...                        
7608    [two, giant, crane, hold, bridg, collaps, near...
7609    [@, aria_ahrari, @, thetawniest, the, control,...
7610    [m1.94, [, 01:04, utc, ], ?, 5km, S, volcano, ...
7611    [polic, investig, e-bik, collid, car, littl, p...
7612    [the, latest, :, more, home, raze, northern, c...
Name: text, Length: 7613, dtype: object

In [58]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [59]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [60]:
from keras.callbacks import EarlyStopping

In [98]:
model.fit(sequences_matrix,train['target'],batch_size=128,epochs=6, validation_split=0.25)

ValueError: Input arrays should have the same number of samples as target arrays. Found 3263 input samples and 7613 target samples.

In [73]:
pred = model.predict(sequences_matrix)

In [74]:
for i in range(len(train)):
    if pred[i] >= 0.5:
        pred[i] = 1
    else:
        pred[i] = 0
pred

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [75]:
right = 0
for i in range(len(train)):
    if pred[i] == train['target'][i]:
        right += 1
print(right/len(train))

0.9138316038355445


In [72]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [76]:
for i in range(len(test)):
    test['text'][i] = word_tokenize(test['text'][i])
print(type(test['text'][0]))
print(test['text'][0])

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'list'>
['Just', 'happened', 'a', 'terrible', 'car', 'crash']


In [77]:
for i in range(len(test)):
    test['text'][i] = [w for w in test['text'][i] if w not in stop_words]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
for i in range(len(test)):
    test['text'][i] = [ps.stem(w) for w in test['text'][i]]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
for i in range(len(test)):
    test['text'][i] = [lemmatizer.lemmatize(w) for w in test['text'][i]]

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(test['text'])
sequences = tok.texts_to_sequences(test['text'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [81]:
predictions = model.predict(sequences_matrix)

In [82]:
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
predictions

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]], dtype=float32)

In [83]:
submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [84]:
submission['target'] = predictions

In [94]:
submission.target = submission.target.astype('int')

In [95]:
type(submission['target'][0])

numpy.int32

In [91]:
for i in range(len(submission)):
    submission['target'][i] = int(submission['target'][i])

c:\users\aayus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [96]:
submission

,id,target
0,0,1
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [97]:
submission.to_csv("sample_sub1.csv", index = False)